### Sampling the solution space of metabolic models

While FBA finds a single maximum for the objective function, one might be interested in the space of viable metabolisms. This space reveals the range of conditions where the cell could live without the assumption of maximizing an objective function. 

The space of viable metabolisms is explored by taking uniform samples from the flux distribution. Besides revealing the cell's metabolic potential, these samples also reveal the conformations of higher likelihood and the burden of regulation needed for the cell to achieve its "maximum growth". In the example below, we compare the point of optimal growth with alternative flux distributions.



### Task: flux sampling 

- Use flux sampling to explore how the three toy species respond to (i) a rich environment, (ii) an environment where the only carbon source is glucose, and (iii) an environment where glucose is absent.


- (given as an example below) For the sugar fermenter, compare biomass production to: (i) glycolysis (reaction rxn00459), (ii) formate production (reaction EX_cpd00047_e), (iii) lactate production (EX_cpd00159_e), and (iv) succinate production (EX_cpd00036_e)

- For the butyrate producer, compare biomass production to: (i) glycolysis (reaction rxn00459), (ii) lactate dehydrogenase (reaction rxn00499), and (iii) butyrate production (reaction EX_cpd00211_e)

- For the acetogen, compare biomass production to: (i) glycolysis (reaction rxn00459), (ii) the Wood-Ljungdahl pathway (reaction rxn39948), and (iii) acetate production (reaction EX_cpd00029_e)

 


#### Example of solution for the sugar fermenter

**step 1** : import packages and create the auxiliary function that allows us to modify the environment

In [ ]:
# import the necessary packages and functions
import os
import cobra
import numpy as np
from cobra.sampling import ACHRSampler

#for making the plots
import matplotlib.pyplot as plt
plt.style.use('seaborn-bright')

# an auxiliary function
def apply_environment(mdl, media_dict):
    '''
    replace the availability of metabolites from the 
    extracellular environment.
    
    mdl: cobrapy.model
    media_dict: [exchangeReactionId]:positive concentration
    
    returns: the objective value on the new environment
    '''
    for i in media_dict:
        if mdl.reactions.has_id(i): #only if model contains the reaction
            mdl.reactions.get_by_id(i).lower_bound=-media_dict[i]
        
    sol_m=mdl.optimize()
    return sol_m.objective_value

**step 2** : load the model, apply the rich media

In [ ]:
#folder with models
modelFolder = '/data/precomputed/sbmlModels'


model_sf = cobra.io.read_sbml_model(os.path.join(modelFolder,'sugar_fermenter_toy_model.xml' ))

rich_media = {'EX_cpd00027_e':10,
         'EX_cpd00011_e':10,
         'EX_cpd11640_e':10,
         'EX_cpd00159_e':10,
         'EX_cpd00047_e':10,
         'EX_cpd00029_e':10,
         'EX_cp00067_e':10
         }

apply_environment(model_sf, rich_media)


**step 3**: perform sampling

In [ ]:
sol = model_sf.optimize()
z = sol.objective_value #the objective value

#reaction fluxes under the optimal solution
glycolysis = model_sf.reactions.rxn00459.flux 
fp = model_sf.reactions.EX_cpd00047_e.flux #formate production
lp = model_sf.reactions.EX_cpd00159_e.flux #lactate production
sp = model_sf.reactions.EX_cpd00036_e.flux #succinate production

achr = ACHRSampler(model_sf, thinning=10) #see https://cobrapy.readthedocs.io/en/latest/sampling.html
s1 = achr.sample(5000) #samples
v = achr.validate(s1) 
sample = s1[v=='v'] #keep only the feasible samples

**step 4**: make plots

In [ ]:
fig, ax = plt.subplots()

#sampled biomass vs the sampled glycolysis
ax.scatter(sample.biomass, sample.rxn00459, s=0.1, color='#add8e6', alpha=0.5)

#optimal biomass vs the optimal glycolysis
ax.scatter(sol.objective_value, glycolysis, s=80, color='#add8e6', label = 'Glycolysis', edgecolors='k')

#sampled biomass vs the sampled formate production
ax.scatter(sample.biomass, sample.EX_cpd00047_e, s=0.1, color='purple', alpha=0.5)

#optimal biomass vs the optimal formate production
ax.scatter(sol.objective_value, fp, s=80, color='purple', label = 'Formate prod', edgecolors='k')

#sampled biomass vs the sampled succinate production
ax.scatter(sample.biomass, sample.EX_cpd00036_e, s=0.1, color='orange', alpha=0.5)

#optimal biomass vs the optimal succinate production
ax.scatter(sol.objective_value, sp, s=80, color='orange', label = 'Succinate prod', edgecolors='k')

#sampled biomass vs the sampled lactate production
ax.scatter(sample.biomass, sample.EX_cpd00159_e, s=0.1, color='#00ff9f', alpha=0.5)

#optimal biomass vs the optimal lactate production
ax.scatter(sol.objective_value, lp, s=80, color='#00ff9f', label = 'Lactate prod', edgecolors='k')

ax.set_ylabel('Flux', fontsize=14)
ax.set_xlabel('Biomass flux', fontsize=14)
plt.legend(fontsize='large', fancybox=True, title='Carb. Ferm.', title_fontsize='x-large', loc=2)
plt.xlim(0,z+1)

plt.savefig('/home/daniel_garza/plots/carbF_sample.png', dpi=300)
